In [2]:
import pandas as pd

In [13]:
import os

In [20]:
diretorio_atual = os.path.abspath('__file__')
diretorio_raiz = os.path.dirname(diretorio_atual)
caminho_dados = os.path.join(diretorio_raiz, 'dados')


In [26]:
for arquivo in os.listdir(caminho_dados):
    caminho_arquivo = os.path.join(caminho_dados, arquivo)
    if 'CARRETA' in arquivo:
        df_carreta = pd.read_excel(caminho_arquivo)
    elif 'magazine' in arquivo:
        df_esl = pd.read_excel(caminho_arquivo)
    elif 'Mobile' in arquivo:
        df_mobile = pd.read_excel(caminho_arquivo)
    elif 'Bipe_Produtos' in arquivo:
        df_bipe = pd.read_excel(caminho_arquivo)
    elif 'Bipe_de_notas' in arquivo:
        df_bipe_notas = pd.read_excel(caminho_arquivo, sheet_name='Plan1')

In [30]:
dfs = [df_carreta, df_esl, df_mobile, df_bipe, df_bipe_notas ]

In [32]:
df_carreta[['PEDIDO', 'NF`s','CHAVE', 'DATA ENTRADA', 'TIPO_FLUXO' ]]

,PEDIDO,NF`s,CHAVE,DATA ENTRADA,TIPO_FLUXO
0,1115127149,216154,52251147960950110733550070002161541075020770,2025-11-25,Porta a Porta
1,1115060437,216151,52251147960950110733550070002161511083900154,2025-11-25,Porta a Porta
2,1115089030,216153,52251147960950110733550070002161531042001699,2025-11-25,Porta a Porta
3,1115058937,216150,52251147960950110733550070002161501051581781,2025-11-25,Porta a Porta
4,1115071254,216152,52251147960950110733550070002161521088820709,2025-11-25,Porta a Porta
...,...,...,...,...,...
11543,1129895302,330021,52260147960950110733550070003300211024609233,2026-01-09,Porta a Porta
11544,1129863780,330009,52260147960950110733550070003300091036543106,2026-01-09,Porta a Porta
11545,1129838244,329992,52260147960950110733550070003299921089790143,2026-01-09,Porta a Porta
11546,1129831414,329989,52260147960950110733550070003299891049521987,2026-01-09,Porta a Porta


In [33]:
df_carreta = df_carreta[['PEDIDO', 'NF`s','CHAVE', 'DATA ENTRADA', 'TIPO_FLUXO' ]]

In [36]:
df_final = (pd.merge(df_carreta, df_esl[['Nota Fiscal/Chave NF-e',
                 'Última ocorrência/Observações',
                 'Pessoa/Nome',
                 'Última ocorrência/Data ocorrência']], left_on='CHAVE', 
                 right_on='Nota Fiscal/Chave NF-e', how='left').drop(columns=['Nota Fiscal/Chave NF-e'])
                 .merge(df_mobile[['Pedido', 'Tipo', 'Entregador']], left_on='PEDIDO', right_on='Pedido', how='left').drop(columns=['Pedido'])
                 .merge(df_bipe_notas[['NF', 'BIPE_NOTAS']], left_on='NF`s', right_on='NF', how='left').drop(columns=['NF'])
                 .merge(df_bipe[['PEDIDO_BIPE', 'BIPE_PRODUTO']], left_on='PEDIDO', right_on='PEDIDO_BIPE', how='left').drop(columns=['PEDIDO_BIPE'])
                 )

In [37]:
df_final_2 = pd.merge(df_final, df_esl[['Nota Fiscal/Chave NF-e','Ocorrência/Ocorrência']], left_on='CHAVE', right_on='Nota Fiscal/Chave NF-e', how='left').drop(columns=['Nota Fiscal/Chave NF-e'])

In [40]:
df_final_2['Última ocorrência/Observações'] = df_final_2['Última ocorrência/Observações'].fillna('Não informado')

In [42]:
filtro = df_final_2['Última ocorrência/Observações'] == 'Não informado'

In [43]:
df_final_2.loc[filtro, 'Última ocorrência/Observações'] = df_final_2.loc[filtro, 'Ocorrência/Ocorrência']

In [45]:
df_final_2 = df_final_2.drop(columns=['Ocorrência/Ocorrência'])

In [46]:
df_final_2.to_excel('Relatorio.xlsx', sheet_name='Relatorio_final', index=False)